# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import r2_score, mean_squared_error, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report

import re

import nltk
from nltk.corpus import stopwords
nltk.download(['stopwords','punkt', 'wordnet'])
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

import pickle
%matplotlib inline

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
def load_data():
    # load data from database
    engine = create_engine('sqlite:///InsertDatabaseName.db')
    df = pd.read_sql_table('DisasterResponse', engine)
    #keep/drop genre?
    
    #remove 2 from 'related'
    df = df.loc[df['related'] != 2]
    
    X = df.message.values
    
    y_data = df.drop(columns = ['id', 'message', 'original', 'genre'])
    y = y_data.values
    target_names = y_data.columns.tolist()
    
    return X, y, target_names

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize
#     stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
#     tokens = [word for word in tokens if word not in stop_words]
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).strip()
#         if clean_tok not in stop_words:
        clean_tokens.append(clean_tok)

    return(clean_tokens)

In [ ]:
# tokenize(df['message'][0])

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [18]:
def build_model():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ]))
        ])),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
    
    parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        'features__text_pipeline__vect__max_features': (None, 5000, 10000),
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'features__text_pipeline__vect__stop_words': (None, 'english'),
        'clf__estimator__n_estimators': [50, 100, 200],
        'clf__estimator__min_samples_split': [2, 3, 4]
        }
    cv = GridSearchCV(pipeline, param_grid=parameters)
    
    return cv

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [19]:
def display_results(cv, y_test, y_pred, target_names):
    target_names = target_names
    cl_report = classification_report(y_test, y_pred, target_names = target_names)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", target_names)
    print("Classification Report:\n", cl_report)
    print("Accuracy:", accuracy)
    print("\nBest Parameters:", cv.best_params_)

In [7]:
def main():
    X, y, target_names = load_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    model = build_model()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    display_results(model, y_test, y_pred, target_names)
    return model

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
main()

Labels: ['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']
Classification Report:
                         precision    recall  f1-score   support

               related       0.82      0.93      0.87      5001
               request       0.83      0.39      0.53      1093
                 offer       0.00      0.00      0.00        29
           aid_related       0.75      0.53      0.62      2735
          medical_help       0.57      0.07      0.13       543
      medical_products       0.62      0.07      0.12       300
     search_and_rescue       0.80   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
# X, y, target_names = load_data()
# X_train, X_test, y_train, y_test = train_test_split(X, y)
# model = model_pipeline()
# model.fit(X_train, y_train)
# y_pred = model.predict(X_test)

# (y_pred == y_test).mean()

0.9420197052746615

In [13]:
# pipeline = model_pipeline()
# parameters = {
#         'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
#         'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
#         'features__text_pipeline__vect__max_features': (None, 5000, 10000),
#         'features__text_pipeline__tfidf__use_idf': (True, False),
#         'features__text_pipeline__vect__stop_words': (None, 'english'),
#         'clf__estimator__n_estimators': [50, 100, 200],
#         'clf__estimator__min_samples_split': [2, 3, 4]
# }
# cv = GridSearchCV(pipeline, param_grid=parameters)

In [14]:
# model = cv

# model.fit(X_train, y_train)
# y_pred = model.predict(X_test)

In [ ]:
# pipeline.get_params

In [17]:
cv.best_params_

{'features__text_pipeline__vect__stop_words': 'english'}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [21]:
model

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'features__text_pipeline__vect__stop_words': (None, 'english')},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [22]:
filename = 'disaster_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
def export_model(model):
    filename = 'disaster_model.sav'
    pickle.dump(model, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.